In [ ]:
import json
import re
import random
from tqdm import tqdm
from typing import Optional, Dict, Sequence, List
# import argparse

def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0
    
def check(key, truth, predict):
    
    if 'print' in predict or 'return' in predict or 'define' in predict:
        return False

    
    if key in ['cycle', 'connectivity', 'hamilton', 'substructure', 'bipartite']:
        if '###' in predict:
            if 'yes' in truth.lower() and 'yes' in predict.split('###')[-1].lower():
                # correct_samples[key].append(v)
                return True
            elif 'no' in truth.lower() and 'no' in predict.split('###')[-1].lower():
                return True
            return False
        else:
            matches = re.findall(r'(yes|no)', predict, flags=re.IGNORECASE)
            if matches:
                last_match = matches[-1].lower()
                if last_match == 'yes' and 'yes' in truth.lower():
                    return True
                elif last_match == 'no' and 'no' in truth.lower():
                    return True
                return False
            else:
                return False
                      
    elif key in ['flow', 'shortest', 'triplet']:
      
        t_num = extract_last_num(truth)
        p_num = extract_last_num(predict.split('###')[-1])
        if abs(t_num - p_num) < 1e-2:
            return True
        return False
                
    elif key == 'topology':
        
        if '###' in predict:
            pre = predict.split('###')[-1].strip(' ')
            truth = truth.split('###')[-1].strip(' ')
            if truth in pre or pre in truth:
                return True
            return False
        else:
            truth = truth.split('###')[-1].split(',')
            for t in truth:
                if t in predict or t.strip(' ') in predict:
                    return True
            return False
    

In [22]:
import json
with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_nlgtest_augreverse_output.json') as f:
    datas = f.readlines()
print(len(datas))

tasks = ['cycle', 'connectivity', 'hamilton', 'substructure', 'bipartite', 'flow', 'shortest', 'triplet', 'topology']

samples = {0:{task:0 for task in tasks}, 4:{task:0 for task in tasks}, 2:{task:0 for task in tasks}}
allsamples = {0:{task:[] for task in tasks}, 4:{task:[] for task in tasks}, 2:{task:[] for task in tasks}}
correct_samples  = {task:[] for task in tasks}
all_samples  = {task:[] for task in tasks}
temp = 0
select_samples = []
math_samples = []
for data in datas:
    
    data = json.loads(data)
        
    task = data['task']
    
    if task in ['3','4','5', '6', '7', '8','9']:
        select_samples.append(data)
        continue
    if "X" in data:
        temp += 1
        math_samples.append(data)
        continue
    if task in tasks:
        task = data['task']
        # print('here')
        if check(task, data['response'], data['dv3_response']) and len(data['dv3_response']) >=15:
            if data['query'] not in  correct_samples[task]:
                correct_samples[task].append(data['query'])
                samples[data['shot']][task] += 1
            #     select_samples.append(data)
            
                
        if data['query'] not in all_samples[task]:
            all_samples[task].append(data['query'])
        allsamples[data['shot']][task].append(data)
        
# with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/train_set_v4/gpt4_topology_v4.json', 'w' ) as writer:
#     for sample in select_samples:
#         writer.write(json.dumps(sample, ensure_ascii=False) + '\n')


# with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_math_reverse_v4.json', 'w' ) as writer:
#     for sample in math_samples:
#         writer.write(json.dumps(sample, ensure_ascii=False) + '\n')
# with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_math_v4.json', 'w' ) as writer:
#     for sample in select_samples:
#         writer.write(json.dumps(sample, ensure_ascii=False) + '\n')

72838


In [ ]:
def check_zero(key, truth, predict):
    
    if 'print' in predict or 'return' in predict or 'define' in predict:
        return False

    
    if key in ['cycle', 'connectivity', 'hamilton', 'substructure', 'bipartite']:
        matches = re.findall(r'(yes|no)', predict, flags=re.IGNORECASE)
        if matches:
            last_match = matches[-1].lower()
            if last_match == 'yes' and 'yes' in truth.lower():
                return True
            elif last_match == 'no' and 'no' in truth.lower():
                return True
            return False
        else:
            return False
                      
    elif key in ['flow', 'shortest', 'triplet']:
      
        t_num = extract_last_num(truth)
        p_num = extract_last_num(predict.split('###')[-1])
        if abs(t_num - p_num) < 1e-2:
            return True
        
        p_num = extract_last_num(predict.split('###')[0])
        if abs(t_num - p_num) < 1e-2:
            return True
        if str(t_num) in predict:
            return True
        
        return False
                
    elif key == 'topology':
        
        if '###' in predict:
            pre = predict.split('###')[-1].strip(' ')
            truth = truth.split('###')[-1].strip(' -> ').strip(' ')
            if truth in pre or pre in truth:
                return True
            return False
        else:
            truth = truth.split('###')[-1].split(',')
            for t in truth:
                if t in predict or t.strip(' ') in predict:
                    return True
            return False
    

for shot, sample in allsamples.items():
    
    # for sample in refine_datas:
    with open(f'/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/{shot}_errors.json','w') as writer:
        
        for task, value in sample.items():
            print(task)
            temp = 0
            for  v in value:
                # print(v['response'])
                # print(v['dv3_response'])
                # break
                
                if check(task, v['response'], v['dv3_response']):
                    temp +=1
                else:
                    writer.write(json.dumps(v, ensure_ascii=False) + '\n')
            print(temp/len(value))


In [23]:
## count distribution on different graph edges
  
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count
total_es, total_vs =0 ,0

for task, value in allsamples[2].items():
    print(task)
    temp = 0
    new_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    all_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    for  v in value:

        smallest_number, largest_number, parentheses_count = find_range(query)
        new_data['v_range'] = [smallest_number, largest_number]
        new_data['edegs'] = parentheses_count - 1

        query = v['query']

        if new_data['v_range'][1] < 10:
            all_data[10] += 1
            
        elif new_data['v_range'][1] <20:
            all_data[20] +=1

        elif new_data['v_range'][1] <30:
            all_data[40] += 1

        elif new_data['v_range'][1] <40:
            all_data[60] += 1
        elif new_data['v_range'][1] < 50:
            all_data[80] += 1
        else:
            all_data[100] += 1

        if not check(task, v['response'], v['dv3_response']):
            temp +=1
        else:
            
            if new_data['v_range'][1] < 10:
                new_data[10] += 1
                
            elif new_data['v_range'][1] <20:
                new_data[20] +=1

            elif new_data['v_range'][1] <30:
                new_data[40] += 1

            elif new_data['v_range'][1] <40:
                new_data[60] += 1
            elif new_data['v_range'][1] < 50:
                new_data[80] += 1
            else:
                new_data[100] += 1

    for key, value in new_data.items():
        try:
            print(key, value/all_data[key]*100)
        except:
            print(key)


cycle
10 90.47619047619048
20 75.0
40 51.21951219512195
60 66.66666666666666
80 45.94594594594595
100 38.07106598984771
v_range
edegs
connectivity
10 91.42857142857143
20 94.0
40 82.92682926829268
60 72.22222222222221
80 65.0
100 43.43434343434344
v_range
edegs
hamilton
10 45.294117647058826
20 47.023809523809526
40 53.086419753086425
60 68.42105263157895
80 79.59183673469387
100 100.0
v_range
edegs
substructure
10 43.08510638297872
20 52.4390243902439
40 35.46099290780142
60
80 0.0
100
v_range
edegs
bipartite
10 84.375
20 63.0
40 34.24657534246575
60 36.734693877551024
80 65.55555555555556
100 90.66666666666666
v_range
edegs
flow
10 12.5
20 2.5
40 1.744186046511628
60 3.289473684210526
80 3.4013605442176873
100 100.0
v_range
edegs
shortest
10 31.48148148148148
20 26.785714285714285
40 13.88888888888889
60 15.476190476190476
80 4.49438202247191
100 4.381443298969072
v_range
edegs
triplet
10 44.94047619047619
20 14.285714285714285
40 4.964539007092199
60
80
100 100.0
v_range
edegs
topol

In [ ]:
for k, value in samples.items():
    print(k)
    for key, v in value.items():
        print(key, v/400*100)

In [21]:
## count distribution on different graph edges of mistral
  
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count
total_es, total_vs =0 ,0

allsamples = {}

tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]

for task in tasks:
    with open(f'/cpfs/user/chennuo/CN/output/deepspeed/nlgreasoning/mistral_dpo_beta0.5/torch.bfloat16/test/_gen_{task}_datas.jsonl') as f:
        datas = f.readlines()
    datas = [json.loads(data) for data in datas]
    allsamples[task] = datas

for task, value in allsamples.items():
    print(task)
    temp = 0
    new_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    all_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    

    for  v in value:
        smallest_number, largest_number, parentheses_count = find_range(v["source_data"]['input_prompt'])
        new_data['v_range'] = [smallest_number, largest_number]
        new_data['edegs'] = parentheses_count - 1

        if new_data['v_range'][1] < 10:
            all_data[10] += 1
            
        elif new_data['v_range'][1] <20:
            all_data[20] +=1

        elif new_data['v_range'][1] <30:
            all_data[40] += 1

        elif new_data['v_range'][1] <40:
            all_data[60] += 1
        elif new_data['v_range'][1] < 50:
            all_data[80] += 1
        else:
            all_data[100] += 1

            
        if not check(task, v["source_data"]['answer'], v['output_str']):
            temp +=1
        else:
            
            # new_data['v_range'] = v['source_data']["node_range"]
            if new_data['v_range'][1] < 10:
                new_data[10] += 1
                
            elif new_data['v_range'][1] <20:
                new_data[20] +=1

            elif new_data['v_range'][1] <30:
                new_data[40] += 1

            elif new_data['v_range'][1] <40:
                new_data[60] += 1
            elif new_data['v_range'][1] < 50:
                new_data[80] += 1
            else:
                new_data[100] += 1

    for key, value in new_data.items():
        try:
            print(key, value/all_data[key]*100)
        except:
            print(key)


connectivity
10 80.0
20 88.0
40 53.65853658536586
60 61.111111111111114
80 67.5
100 54.04040404040404
v_range
edegs
cycle
10 73.80952380952381
20 56.81818181818182
40 85.36585365853658
60 79.48717948717949
80 80.55555555555556
100 58.080808080808076
v_range
edegs
flow
10 44.047619047619044
20 45.0
40 27.906976744186046
60 32.89473684210527
80 27.027027027027028
100
v_range
edegs
bipartite
10 65.625
20 56.00000000000001
40 91.66666666666666
60 85.71428571428571
80 75.55555555555556
100 97.87234042553192
v_range
edegs
hamilton
10 52.94117647058824
20 59.523809523809526
40 72.8395061728395
60 72.36842105263158
80 67.56756756756756
100
v_range
edegs
triplet
10 56.547619047619044
20 11.801242236024844
40 2.8169014084507045
60
80
100
v_range
edegs
shortest
10 70.37037037037037
20 19.642857142857142
40 25.0
60 7.142857142857142
80 9.090909090909092
100 2.051282051282051
v_range
edegs
topology
10 59.34065934065934
20 85.71428571428571
40 96.51162790697676
60 96.29629629629629
80 92.30769230769

In [5]:
## count distribution for test set
  
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count


allsamples = {}

tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]


tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
all_datas = []

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

import json
ind = 0
alls = 0
for task in tasks:
    with open(f"/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/dpo_datas/{task}_train_source_rfts.json", "r") as f:
        datas = json.load(f)
    print(task, len(datas))
    ind += len(datas)
    task_specific = 0
    total_vs = 0
    for key, value in datas.items():      
        if "CoT_response" in value and len(value["neg_rft_paths_5"]) >=1:
            responses = value["CoT_response"]
            new_data = dict()
            new_data['prompt'] = PROMPT_DICT["prompt_no_input"].format(instruction=value['query'].replace('Q:','\nQ:'))
            new_data['chosen'] = responses
            new_data['rejected'] = value["neg_rft_paths_5"][0]
            all_datas.append(new_data)
            task_specific += 1
        # break
      
# with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/dpo_datas/dpo_train_dsformat_V1.json', 'w') as w:
#     for sample in all_datas:
#         w.write(json.dumps(sample, ensure_ascii=False) + '\n')
# print(len(all_datas))
            smallest_number, largest_number, parentheses_count = find_range(value['query'])
            new_data['v_range'] = [smallest_number, largest_number]
            new_data['edegs'] = parentheses_count - 1

         
            total_vs += largest_number - smallest_number

            
    print(task, total_vs)
    alls+=total_vs
print(alls)

connectivity 972
connectivity 20705
cycle 982
cycle 22132
flow 545
flow 12863
bipartite 689
bipartite 11546
hamilton 962
hamilton 24846
triplet 551
triplet 5842
shortest 490
shortest 21670
topology 500
topology 12802
substructure 965
substructure 10050
142456


In [ ]:
## count dpo distribution on different graph edges
  
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count
total_es, total_vs =0 ,0

for task, value in allsamples[2].items():
    print(task)
    temp = 0
    new_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    all_data={10:0, 20:0, 40:0, 60:0, 80:0, 100:0}
    for  v in value:

        smallest_number, largest_number, parentheses_count = find_range(query)
        new_data['v_range'] = [smallest_number, largest_number]
        new_data['edegs'] = parentheses_count - 1

        query = v['query']

        if new_data['v_range'][1] < 10:
            all_data[10] += 1
            
        elif new_data['v_range'][1] <20:
            all_data[20] +=1

        elif new_data['v_range'][1] <30:
            all_data[40] += 1

        elif new_data['v_range'][1] <40:
            all_data[60] += 1
        elif new_data['v_range'][1] < 50:
            all_data[80] += 1
        else:
            all_data[100] += 1

        if not check(task, v['response'], v['dv3_response']):
            temp +=1
        else:
            
            if new_data['v_range'][1] < 10:
                new_data[10] += 1
                
            elif new_data['v_range'][1] <20:
                new_data[20] +=1

            elif new_data['v_range'][1] <30:
                new_data[40] += 1

            elif new_data['v_range'][1] <40:
                new_data[60] += 1
            elif new_data['v_range'][1] < 50:
                new_data[80] += 1
            else:
                new_data[100] += 1

    for key, value in new_data.items():
        try:
            print(key, value/all_data[key]*100)
        except:
            print(key)


In [ ]:


import json
with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/gpt4_generate_nlg_v2.json') as f:
    datas = f.readlines()
refine_datas = []
temp = 0
for data in datas:
    data = json.loads(data)
    new_data = dict()
    new_data['query'] = data['query']
    new_data['task'] = data['task']
    new_data['response'] = data['response']
    response = data['dv3_response'].split('A:')[-1].split('\n\n')[-1]
    if '###' not in response:
        response +=  data['response'] + '.'
    if 'print' in response or 'return' in response or 'def' in response:
        temp += 1
        continue
    new_data['CoT_response'] = response
    refine_datas.append(new_data)

with open('/cpfs/user/chennuo/CN/Graph_RFT_Data/gpt4data/graph_source_data_v2.json', 'w' ) as writer:
    for sample in refine_datas:
        writer.write(json.dumps(sample, ensure_ascii=False) + '\n')

In [ ]:
import random
random.sample(refine_datas, 10)[0]

In [ ]:

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/graph_v2_dsformat.json', 'w' ) as writer:
    for sample in refine_datas:
        new_sample = dict()
        new_sample['prompt'] = PROMPT_DICT["prompt_no_input"].format(instruction=sample['query'])
        new_sample['chosen'] = sample['CoT_response']
        new_sample['reject'] = 'I do not know'
        writer.write(json.dumps(new_sample, ensure_ascii=False) + '\n')

In [ ]:
import re

text = "This is a sample string with YES and no, as well as YES."

# Use regular expressions to find the last 'yes' or 'no' (case-insensitive)
matches = re.findall(r'(yes|no)', text, flags=re.IGNORECASE)

if matches:
    last_match = matches[-1].lower()
    if last_match == 'yes':
        print('yes')
    elif last_match == 'no':
        print('no')
else:
    print("No 'yes' or 'no' found in the text.")


In [ ]:
import os
import json
import random

# Define the paths
input_folder = '/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/train_set'
output_folder = '/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/train_set_shuffle'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get the list of JSON files in the input folder
json_files = [file for file in os.listdir(input_folder) if file.endswith('.json')]

# Iterate over each JSON file
for file in json_files:
    # Read the JSON file
    with open(os.path.join(input_folder, file)) as f:
        datas = f.readlines()
    
    data = [json.loads(item) for item in datas]
    # Assign sample IDs
    max_length = len(data)
    for i, sample in enumerate(data):
        sample['sample_id'] = i
    
    # Shuffle the samples
    random.shuffle(data)
    
    # Write the shuffled data to a new JSON file in the output folder
    output_file = os.path.join(output_folder, file)
    with open(output_file, 'w') as f:
        # json.dump(data, f)
        for new_sample in data:
            f.write(json.dumps(new_sample, ensure_ascii=False) + '\n')


In [ ]:
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count

for task in tasks:
    
    with open(f'/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/rft_datas/{task}_train_source_rfts.json') as f:
        datas = json.load(f)
    
    task_dicts = {}
    i = 0
    choices = 0
    for key, value in datas.items():
        
        # if 'CoT_response' in value and len(value['neg_rft_paths_5']) >=2:
        if len(value['neg_rft_paths_5']) >=1:
            new_data = dict()
            new_data['task'] = value['task']
            new_data['query'] = value['query']
            new_data['answer'] = value['answer']
            new_data['response'] = value['CoT_response'] if 'CoT_response' in value else value['pos_rft_paths_5'][-1]
            new_data['neg_response'] = value['neg_rft_paths_5']
            smallest_number, largest_number, parentheses_count = find_range(value['query'])
            new_data['v_range'] = [smallest_number, largest_number]
            new_data['edegs'] = parentheses_count - 1
            if new_data['v_range'][1] < 5:
                new_data['range'] = 5
                
            if new_data['v_range'][1] < 10:
                new_data['range'] = 10
            elif new_data['v_range'][1] < 15:
                new_data['range'] = 15
                
            elif new_data['v_range'][1] <20:
                new_data['range'] = 20
                
            elif new_data['v_range'][1] <30:
                new_data['range'] = 30
                
            elif new_data['v_range'][1] <40:
                new_data['range'] = 40
                
            elif new_data['v_range'][1] <50:
                new_data['range'] = 50
                
            elif new_data['v_range'][1] <60:
                new_data['range'] = 60
                
            elif new_data['v_range'][1] <70:
                new_data['range'] = 70
                
            elif new_data['v_range'][1] < 80:
                new_data['range'] = 80
                
            elif new_data['v_range'][1] < 90:
                new_data['range'] = 90
            else:
                new_data['range'] = 100
            task_dicts[i] = new_data
            i += 1
            choices += 1 + len(new_data['neg_response']) 
    with open(f'/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/prob_data/{task}_prob.json', 'w') as f: 
        json.dump(task_dicts, f, ensure_ascii=False, indent=4)
    print(task, i, choices/i )

In [ ]:
import json
with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/prob_data/shortest_prob.json') as f:
    datas = json.load(f)

# for data in datas:
counts = {item*10:0 for item in range(1,11)}
for key, value in datas.items():
    if value['v_range'][1] <10:
        counts[10] += 1
    elif value['v_range'][1] <20:
        counts[20] += 1
    elif value['v_range'][1] <30:
        counts[30] += 1
    elif value['v_range'][1] <40:
        counts[40] += 1
    elif value['v_range'][1] <50:
        counts[50] += 1
    elif value['v_range'][1] <60:
        counts[60] += 1
    elif value['v_range'][1] <70:
        counts[70] += 1
    elif value['v_range'][1] <80:
        counts[80] += 1
    elif value['v_range'][1] <90:
        counts[90] += 1
    else:
        counts[100] += 1
        
print(counts)       

In [ ]:
value['query']

In [ ]:
import json
tasks = ['connectivity', 'cycle','flow', 'bipartite', 'hamilton', 'triplet', 'shortest', 'topology', 'substructure',]
import re

def extract_all_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        # num_str = res[-1][0]
        # return float(num_str)
        return [int(item[0]) for item in res]
    else:
        return None


def find_range(string):
    # Updated input string with the full provided text
    updated_input_string = string.split('Q:')[-1].split('The notes are numbered from')[-1].split(',')[0]
    # Extracting numbers and counting parentheses
    # numbers = [int(num) for num in updated_input_string if num.isdigit()]
    try:
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    except:
        updated_input_string = string.split('Q:')[-1].split('The nodes of graph G are numbered from')[-1].split(',')[0]
        
        numbers = extract_all_num(updated_input_string)
        smallest_number = min(numbers)
        largest_number = max(numbers)
    # Finding the smallest and largest number
    
    # Counting the parentheses
    parentheses_count = string.count("(")  # Counting either "(" or ")" would give the same result

    return smallest_number, largest_number, parentheses_count
total_es, total_vs =0 ,0
for task in tasks:
    
    with open(f'/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/datasets/data/rft_datas/{task}_train_source_rfts.json') as f:
        datas = json.load(f)
    
    task_dicts = {}
    i = 0
    vs, es = 0, 0 
    for key, value in datas.items():
        responses = []
        
        if "CoT_response" in value:
            responses.append(value["CoT_response"])
        responses += value["pos_rft_paths_5"]
        responses = list(set(responses))
        for ans in responses[:5]:
            smallest_number, largest_number, parentheses_count = find_range(value['query'])
        vs += largest_number - smallest_number 
        es += parentheses_count - 1
            # all_datas.append(new_data)
    total_vs += vs
    print(task, 'nodes',vs, 'edge',es)
total_vs      

In [6]:
import json
with open('/cpfs/user/chennuo/CN/Graph-Reasoning-LLM/NLGraph/topology/test.json') as f:
    data = json.load(f)
for key, value in data.items():
    if value['difficulty'] == 'easy':
        print(value)

{'question': 'In a directed graph with 10 nodes numbered from 0 to 9:\nnode 0 should be visited before node 7\nnode 0 should be visited before node 9\nnode 2 should be visited before node 9\nnode 3 should be visited before node 4\nnode 3 should be visited before node 0\nnode 3 should be visited before node 1\nnode 3 should be visited before node 5\nnode 3 should be visited before node 9\nnode 4 should be visited before node 6\nnode 4 should be visited before node 0\nnode 4 should be visited before node 7\nnode 4 should be visited before node 8\nnode 4 should be visited before node 1\nnode 4 should be visited before node 2\nnode 5 should be visited before node 2\nnode 6 should be visited before node 0\nnode 6 should be visited before node 7\nnode 6 should be visited before node 8\nnode 6 should be visited before node 2\nnode 6 should be visited before node 9\nnode 7 should be visited before node 8\nnode 7 should be visited before node 9\nQ: Can all the nodes be visited? Give the solutio